# Fraud Data Preprocessing Pipeline (from raw data to model-ready dataframe)

#### Creating a synthetic dataset to simulate the fraud detection problem and data preprocessing pipeline.

In this approach we will conduct a real world fraud detection tests simulation. For this, we're using a synthetic dataset that mimics real-world bank transactions.

The dataset includes demographic, financial, and behavioral attributes, along with a binary target variable (1 = fraud, 0 = legitimate).

Data preprocessing includes:

- Handling missing values
- Removing outliers
- Normalizing numeric features
- Encoding categorical variables
- Creating and selecting the most relevant features (Feature Engineering)

These steps ensure data quality and improve model performance.

## 1. Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

## 2. Simulating a raw dataset

To create a fake raw dataset that simulates real-world bank transactions, we will use numpy and pandas.

In [9]:
np.random.seed(42)  # for reproducibility

n = 10000  # number of samples
raw_df = pd.DataFrame({
    'transaction_id': np.arange(n),
    'country': np.random.choice(['Brazil', 'USA', 'UK', 'Germany'], n),
    'city': np.random.choice(['Rio', 'New York', 'London', 'Berlin'], n),
    'district': np.random.choice(['Center', 'North', 'South', 'West'], n),
    'zip': np.random.randint(10000, 99999, n),
    'ip': [f"192.168.{np.random.randint(0,255)}.{np.random.randint(0,255)}" for _ in range(n)],
    'datetime': pd.date_range('2024-01-01', periods=n, freq='min'),
    'os': np.random.choice(['Windows', 'Android', 'iOS'], n),
    'value': np.random.exponential(scale=100, size=n).round(2),
    'background_checks': np.random.randint(0, 5, n),
    'complaints': np.random.randint(0, 10, n),
    'transactions': np.random.randint(1, 50, n),
    'credit_score': np.random.normal(650, 100, n).astype(int),
    'credit_limit': np.random.uniform(500, 20000, n).round(2),
    'device': np.random.choice(['mobile', 'desktop'], n),
    'browser': np.random.choice(['Chrome', 'Safari', 'Firefox', 'Edge'], n),
    'is_fraud': np.random.choice([0, 1], n, p=[0.985, 0.015])  # imbalanced target
})

display("Raw dataset sample:")
display(raw_df.sample(5))

'Raw dataset sample:'

,transaction_id,country,city,district,zip,ip,datetime,os,value,background_checks,complaints,transactions,credit_score,credit_limit,device,browser,is_fraud
9339,9339,Brazil,London,North,93048,192.168.176.75,2024-01-07 11:39:00,Windows,111.96,4,1,19,693,9281.39,mobile,Safari,0
7915,7915,USA,Berlin,Center,25948,192.168.241.49,2024-01-06 11:55:00,iOS,3.18,2,1,48,682,3341.21,desktop,Firefox,0
3962,3962,Brazil,Berlin,West,78315,192.168.188.88,2024-01-03 18:02:00,Windows,7.04,1,7,38,716,8277.55,mobile,Edge,0
7723,7723,Brazil,New York,Center,46559,192.168.72.88,2024-01-06 08:43:00,iOS,13.48,2,2,44,762,4688.45,mobile,Safari,0
2932,2932,Brazil,Berlin,South,87903,192.168.47.86,2024-01-03 00:52:00,Windows,20.57,0,3,34,646,15598.18,mobile,Safari,0


## 3. Feature Engineering

To enrich the dataset with additional features, we can perform feature engineering. This can include creating new features from existing data, such as time-based features, transaction amount-based features, or other relevant features.

In this case we will split datetime column into day, hour and minute columns and drop the original datetime column to reduce the dimensionality of the dataset.

In [10]:
# --- 3. Feature extraction from datetime ---
raw_df['day'] = raw_df['datetime'].dt.day
raw_df['hour'] = raw_df['datetime'].dt.hour
raw_df['minute'] = raw_df['datetime'].dt.minute
raw_df.drop(columns=['datetime'], inplace=True)
raw_df.head()

,transaction_id,country,city,district,zip,ip,os,value,background_checks,complaints,transactions,credit_score,credit_limit,device,browser,is_fraud,day,hour,minute
0,0,UK,London,South,35499,192.168.140.92,iOS,121.49,0,3,19,768,6246.51,desktop,Chrome,0,1,0,0
1,1,Germany,Berlin,West,16421,192.168.106.108,Windows,182.55,2,4,20,601,18677.37,desktop,Chrome,0,1,0,1
2,2,Brazil,London,West,62204,192.168.87.249,Windows,36.29,0,9,11,655,14260.93,mobile,Chrome,0,1,0,2
3,3,UK,Berlin,North,34591,192.168.44.107,Windows,1.48,2,1,24,603,2218.27,mobile,Chrome,0,1,0,3
4,4,UK,Berlin,Center,70940,192.168.45.76,iOS,80.12,0,9,22,534,13281.11,desktop,Chrome,0,1,0,4


Creating a **"security index"** feature based on the background checks and complaints and a **"average value"** of transactions.

In [11]:
raw_df['security_index'] = raw_df['background_checks'] * 0.7 + raw_df['complaints'] * 0.3
raw_df['avg_value_per_tx'] = raw_df['value'] / raw_df['transactions']

Checking summary of dataset.

In [12]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     10000 non-null  int64  
 1   country            10000 non-null  object 
 2   city               10000 non-null  object 
 3   district           10000 non-null  object 
 4   zip                10000 non-null  int64  
 5   ip                 10000 non-null  object 
 6   os                 10000 non-null  object 
 7   value              10000 non-null  float64
 8   background_checks  10000 non-null  int64  
 9   complaints         10000 non-null  int64  
 10  transactions       10000 non-null  int64  
 11  credit_score       10000 non-null  int64  
 12  credit_limit       10000 non-null  float64
 13  device             10000 non-null  object 
 14  browser            10000 non-null  object 
 15  is_fraud           10000 non-null  int64  
 16  day                1000

## 4. Explotary data analysis

Handling data types, missing values and outliers is a common task in data preprocessing. As our dataset is synthetic, we injected some **missing values** in the dataset to simulate real-world scenarios.

In [13]:
# Cast zip to string
raw_df['zip'] = raw_df['zip'].astype(str)

# Remove a random fraction of rows for some columns
for col in ['city', 'district', 'os', 'zip']:
    random_percentage = np.random.uniform(0.01, 0.03)
    raw_df.loc[raw_df.sample(frac=random_percentage).index, col] = np.nan

raw_df.isnull().sum()

transaction_id         0
country                0
city                 261
district             276
zip                  223
ip                     0
os                   203
value                  0
background_checks      0
complaints             0
transactions           0
credit_score           0
credit_limit           0
device                 0
browser                0
is_fraud               0
day                    0
hour                   0
minute                 0
security_index         0
avg_value_per_tx       0
dtype: int64

Handling missing values

In [14]:
raw_df['city'] = raw_df['city'].fillna('Unknown')
raw_df['district'] = raw_df['district'].fillna('Unknown')
raw_df['os'] = raw_df['os'].fillna('Unknown')
raw_df['zip'] = raw_df['zip'].fillna('Unknown')

raw_df.isnull().sum()

transaction_id       0
country              0
city                 0
district             0
zip                  0
ip                   0
os                   0
value                0
background_checks    0
complaints           0
transactions         0
credit_score         0
credit_limit         0
device               0
browser              0
is_fraud             0
day                  0
hour                 0
minute               0
security_index       0
avg_value_per_tx     0
dtype: int64

## 5. Variables encoding

Checking the distribution of categorical variables:

In [44]:
cat_cols = ['country', 'city', 'district', 'os', 'device', 'browser']
for col in cat_cols:
    print(f"{col}: {raw_df[col].nunique()} in {raw_df[col].shape[0]}")

country: 4 in 10000
city: 5 in 10000
district: 5 in 10000
os: 4 in 10000
device: 2 in 10000
browser: 4 in 10000


Since we have a few classes, we will use one-hot encoding to encode categorical variables.

In [45]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(raw_df[cat_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols))

encoded_df.head()

,country_Brazil,country_Germany,country_UK,country_USA,city_Berlin,city_London,city_New York,city_Rio,city_Unknown,district_Center,district_North,district_South,district_Unknown,district_West,os_Android,os_Unknown,os_Windows,os_iOS,device_desktop,device_mobile,browser_Chrome,browser_Edge,browser_Firefox,browser_Safari
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [46]:
# Merge encoded columns back
df = pd.concat([raw_df.drop(columns=cat_cols), encoded_df], axis=1)

pd.set_option('display.max_columns', None)
display(df.shape)
display(df.head())

(10000, 39)

,transaction_id,zip,ip,value,background_checks,complaints,transactions,credit_score,credit_limit,is_fraud,day,hour,minute,security_index,avg_value_per_tx,country_Brazil,country_Germany,country_UK,country_USA,city_Berlin,city_London,city_New York,city_Rio,city_Unknown,district_Center,district_North,district_South,district_Unknown,district_West,os_Android,os_Unknown,os_Windows,os_iOS,device_desktop,device_mobile,browser_Chrome,browser_Edge,browser_Firefox,browser_Safari
0,0,35499,192.168.140.92,121.49,0,3,19,768,6246.51,0,1,0,0,0.9,6.394211,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,16421,192.168.106.108,182.55,2,4,20,601,18677.37,0,1,0,1,2.6,9.127500,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2,62204,192.168.87.249,36.29,0,9,11,655,14260.93,0,1,0,2,2.7,3.299091,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,34591,192.168.44.107,1.48,2,1,24,603,2218.27,0,1,0,3,1.7,0.061667,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,4,70940,192.168.45.76,80.12,0,9,22,534,13281.11,0,1,0,4,2.7,3.641818,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


----

## 6. Numerical features standardization and dimensionality reduction

Many machine learning estimators does not perform well when the individual features do not more or less look like standard normally distributed data. As a good practice, we should standardize the numerical features and reduce their dimensionality.

In this approach, we will use the `StandardScaler` from `sklearn.preprocessing` to standardize the numerical features and the `PCA` from `sklearn.decomposition` to reduce features dimensionality.

In [47]:
# Scale numerical features
num_cols = df.drop(columns=['transaction_id', 'zip', 'ip', 'is_fraud']).select_dtypes(include=np.number).columns
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

display(df_scaled.shape)
display(df_scaled.head())

(10000, 39)

,transaction_id,zip,ip,value,background_checks,complaints,transactions,credit_score,credit_limit,is_fraud,day,hour,minute,security_index,avg_value_per_tx,country_Brazil,country_Germany,country_UK,country_USA,city_Berlin,city_London,city_New York,city_Rio,city_Unknown,district_Center,district_North,district_South,district_Unknown,district_West,os_Android,os_Unknown,os_Windows,os_iOS,device_desktop,device_mobile,browser_Chrome,browser_Edge,browser_Firefox,browser_Safari
0,0,35499,192.168.140.92,0.203084,-1.411643,-0.543369,-0.430575,1.201543,-0.712857,0,-1.495605,-1.659421,-1.701871,-1.413656,-0.112908,-0.582893,-0.577504,1.742740,-0.575195,-0.569344,1.772710,-0.570884,-0.581815,-0.12382,-0.559332,-0.570268,1.753089,-0.163705,-0.569190,-0.705304,-0.168474,-0.685049,1.455063,1.016537,-1.016537,1.753562,-0.573193,-0.586896,-0.579044
1,1,16421,192.168.106.108,0.811126,0.004177,-0.197494,-0.359746,-0.469407,1.510107,0,-1.495605,-1.659421,-1.644103,-0.126566,0.004015,-0.582893,1.731589,-0.573809,-0.575195,1.756408,-0.564108,-0.570884,-0.581815,-0.12382,-0.559332,-0.570268,-0.570422,-0.163705,1.756883,-0.705304,5.935645,-0.685049,-0.687256,1.016537,-1.016537,1.753562,-0.573193,-0.586896,-0.579044
2,2,62204,192.168.87.249,-0.645348,-1.411643,1.531879,-0.997205,0.070900,0.720332,0,-1.495605,-1.659421,-1.586334,-0.050855,-0.245308,1.715581,-0.577504,-0.573809,-0.575195,-0.569344,1.772710,-0.570884,-0.581815,-0.12382,-0.559332,-0.570268,-0.570422,-0.163705,1.756883,-0.705304,-0.168474,1.459750,-0.687256,-0.983732,0.983732,1.753562,-0.573193,-0.586896,-0.579044
3,3,34591,192.168.44.107,-0.991991,0.004177,-1.235118,-0.076431,-0.449395,-1.433212,0,-1.495605,-1.659421,-1.528565,-0.807967,-0.383796,-0.582893,-0.577504,1.742740,-0.575195,1.756408,-0.564108,-0.570884,-0.581815,-0.12382,-0.559332,1.753562,-0.570422,-0.163705,-0.569190,-0.705304,-0.168474,1.459750,-0.687256,-0.983732,0.983732,1.753562,-0.573193,-0.586896,-0.579044
4,4,70940,192.168.45.76,-0.208884,-1.411643,1.531879,-0.218089,-1.139788,0.545114,0,-1.495605,-1.659421,-1.470796,-0.050855,-0.230647,-0.582893,-0.577504,1.742740,-0.575195,1.756408,-0.564108,-0.570884,-0.581815,-0.12382,1.787845,-0.570268,-0.570422,-0.163705,-0.569190,-0.705304,-0.168474,-0.685049,1.455063,1.016537,-1.016537,1.753562,-0.573193,-0.586896,-0.579044


In [48]:
# PCA for dimensionality reduction
pca = PCA(n_components=20)
pca_features = pca.fit_transform(df_scaled[num_cols])

df_pca = pd.DataFrame(
    pca_features, 
    columns=[f'pca_{i+1}' for i in range(20)]
    )

# Combine PCA features + target
final_df = pd.concat([df_pca, df_scaled[['value', 'is_fraud']].reset_index(drop=True)], axis=1)

print("\nPreprocessed dataframe sample:")
display(final_df)


Preprocessed dataframe sample:


,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,value,is_fraud
0,-0.671428,2.384285,0.166130,-0.794337,1.957361,1.354013,-1.743959,-0.005643,-0.181439,-1.478795,-0.189217,-0.975101,-1.525467,-1.289536,-0.883806,1.293161,-1.390802,-0.028964,0.319144,0.535073,0.203084,0
1,0.863544,1.279572,0.534368,-0.520691,0.259223,-0.101313,1.301402,-1.574485,-0.770987,0.163682,-0.028839,2.169757,-0.877596,-1.520939,-0.150134,0.498575,-1.322150,4.198688,2.577431,2.413617,0.811126,0
2,-0.895024,-0.972095,0.408360,-1.357932,-1.361304,1.099777,0.510390,-1.768689,0.527156,-0.004611,0.215966,-0.558087,0.021071,-1.652027,1.161755,2.824849,-1.153931,-0.395659,0.913622,0.033904,-0.645348,0
3,-1.780856,-0.406535,-0.781746,-1.002004,-1.183516,0.142199,-0.288984,0.710746,-0.537743,-1.511962,2.454316,1.199204,0.166825,-0.657215,-1.387942,0.590543,-1.628303,-0.279937,0.277378,1.185532,-0.991991,0
4,0.879539,0.943594,-0.891395,-0.634451,1.456009,-0.498341,-1.887608,0.900030,-0.871284,0.202399,2.142220,1.166698,-1.568136,-1.757523,0.108359,0.639796,-0.664426,0.161594,0.939584,-0.347371,-0.208884,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-3.272626,0.697995,1.462324,-1.336004,2.076722,-1.349106,1.384431,-1.055271,-0.382198,-0.495233,0.574002,0.523351,-0.542282,2.284305,-0.981249,-0.701960,1.560206,-0.398124,-0.566325,0.153074,1.113355,0
9996,-1.025218,2.614890,1.143024,1.298115,0.337518,-0.637039,1.619402,2.518042,-0.036125,-1.542511,1.362148,-0.686199,-0.079691,1.010998,-0.673343,-0.054092,0.158904,-0.945541,-0.344079,0.156783,-0.543776,0
9997,-2.539605,0.698052,-0.414624,-0.501739,-0.514716,1.031083,0.781892,-0.213110,0.231349,-1.161355,0.741476,0.544799,-0.426956,3.840834,0.939579,-1.063721,0.717653,4.187816,1.753997,2.330793,-0.319021,0
9998,-0.500749,-1.222346,-1.073083,-1.335034,0.449434,-0.760098,0.043146,0.010107,1.317452,0.177794,-0.965972,2.830174,-0.920368,0.933586,1.786813,-0.833415,-0.477168,0.722694,-0.461358,-0.549694,0.611367,0


## 7. Saving the preprocessed data

Now we can save the preprocessed data to a CSV file.

In [52]:
os.makedirs("data", exist_ok=True)
final_df.to_csv(os.path.join("data", "bank_transactions_processed.csv"), index=False)